참고 url

https://developers.google.com/youtube/v3/docs/search/list?hl=ko#python,-%EC%98%88-1

https://developers.google.com/youtube/v3/docs/videos?hl=ko

https://m.blog.naver.com/doublet7411/221514043955

https://yobro.tistory.com/190


#1 관련 유튜브 동영상들의 지표를 수집

In [1]:
from apiclient.discovery import build
import pandas as pd
import numpy as np
import datetime
import pytz

In [2]:
#종목명과 날짜를 입력하면 데이터 프레임에 저장되는 함수 호출
#종목명, 날짜, video id 리스트를 데이터 프레임에 저장

In [3]:
#날짜별 조건을 만족하는 영상 ID를 추출하는 함수
def stockVideoIDExtract(pd,index,stockName,month,date,resultCnt):
    
    youtube = build("youtube", "v3", developerKey="api key")
    
    AfterMonth = month
    AfterDate = date-1
    BeforeMonth = month
    BeforeDate = date
    
    date_mapping = {1:31,2:31,3:28,4:31,5:30,6:31,7:30,8:31,9:31,10:30,11:31,12:30}
    
    if month < 8 : 
        AfterYear = 2020
        BeforeYear = 2020
    else:
        AfterYear = 2019
        BeforeYear = 2019
    
    if date == 1: 
        if month == 1: 
            AfterMonth = 12
            AfterYear = 2019
            BeforeYear = 2020
        else:
            AfterMonth = month - 1
        AfterDate = date_mapping[month]
        
    #UTC 기준으로 우리나라 시각은 9시간 빠르기 때문에 이걸 반영한 시간 설정을 해야됨
    search_response = youtube.search().list(
        q= stockName+"주식",
        part="id",#snippet
        order = "viewCount",
        publishedAfter = datetime.datetime(AfterYear, AfterMonth, AfterDate, 15, 0, 0, 0).isoformat("T") + "Z",
        publishedBefore = datetime.datetime(BeforeYear, BeforeMonth, BeforeDate, 14, 59, 59, 0).isoformat("T") + "Z",
        maxResults=resultCnt
    ).execute()
    for search_result in search_response.get("items", []):
        #응답 결과가 비디오 정보인 경우 <- 이 경우만 사용함
        if search_result["id"]["kind"] == "youtube#video":
          pd['Stock'][index] = stockName
          pd['Date'][index] = datetime.date(BeforeYear, month, date).strftime('%Y-%m-%d') 
          pd['Video ID'][index] = search_result["id"]["videoId"]
          #pd['Title'][index] = search_result["snippet"]["title"]
        #응답 결과가 채널 정보인 경우 
        elif search_result["id"]["kind"] == "youtube#channel":
          titleList.append("%s (%s)" % (search_result["snippet"]["title"],search_result["id"]["channelId"]))
        #응답 결과가 플레이리스트 정보인 경우
        elif search_result["id"]["kind"] == "youtube#playlist":
          playlists.append("%s (%s)" % (search_result["snippet"]["title"],search_result["id"]["playlistId"]))
        index = index+1

In [4]:
#영상ID를 이용하여 조회수,좋아요/싫어요수,댓글수를 추출해서 데이터프레임에 저장하는 함수
def stockVideoStatExtract(pd1):
    
    pd2 = pd1.copy()
    youtube = build("youtube", "v3", developerKey="api key")

    for i in range(len(pd1)): 
        request = youtube.videos().list(
            part="statistics",
            id = pd1['Video ID'][i]
        )
        
        response = request.execute()
        
        if response['items']==[]:
            pd2['View'][i] = np.NaN
            pd2['Like'][i] = np.NaN 
            pd2['Dislike'][i] = np.NaN 
            pd2['Comment'][i] = np.NaN 
            
        else:
            #반환되는 동영상 지표들 중 각 지표들의 정보를 반환하는지 확인하고 값 저장
            
            #조회수 저장
            if 'viewCount' in response['items'][0]['statistics']: 
                pd2['View'][i] = int(response['items'][0]['statistics']['viewCount']) 
            else:
                pd2['View'][i] = 0
            #좋아요 수 저장    
            if 'likeCount' in response['items'][0]['statistics']: 
                pd2['Like'][i] = int(response['items'][0]['statistics']['likeCount']) 
            else:
                pd2['Like'][i] = 0
            
            #싫어요 수 저장
            if 'dislikeCount' in response['items'][0]['statistics']: 
                pd2['Dislike'][i] = int(response['items'][0]['statistics']['dislikeCount']) 
            else:
                pd2['Dislike'][i] = 0            
            
            #댓글 수 저장
            if 'commentCount' in response['items'][0]['statistics']: 
                pd2['Comment'][i] = int(response['items'][0]['statistics']['commentCount']) 
            else:
                pd2['Comment'][i] = 0
    return pd2
                        

In [15]:
#종목에 대해 날짜별 주식 종목 관련 동영상 조회수 기준 최대 5개 영상 ID 추출하는 예제
stockName = ["카카오"]
#1개월 단위로만 해야합니다.
month = [x for x in range(3,4)]#월 구간(range 범위 확인)
#각 월별로 시작하는 날짜와 끝나는 날짜 구간을 입력(range 범위 확인)
date = [x for x in range(1,11)]#일 구간

df = pd.DataFrame(columns=['Stock','Date','Video ID','View','Like','Dislike','Comment'])

videosTitles = []  
videosID = []
index = 0
dayResult = 10

for i in range(len(stockName)*len(month)*len(date)*dayResult):
    df= df.append({'Stock' : '' , 'Date' : '', 'Video ID': '','View' : 0 , 'Like' : 0, 'Dislike' : 0, 'Comment' : 0} , ignore_index=True)

df = df.astype({'View': np.int64,'Like': np.int64,'Dislike': np.int64,'Comment': np.int64})
for i in stockName:
    for j in month:
        for k in date:
            stockVideoIDExtract(df,index,i,j,k,dayResult)
            YoutubeStatData = stockVideoStatExtract(df)
            index = index + dayResult

<ipython-input-4-57fbd4323e3b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd2['View'][i] = np.NaN
c:\users\82102\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-4-57fbd4323e3b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd2['Like'][i] = np.NaN
<ipython-input-4-57fbd4323e3b>:1

In [16]:
YoutubeStatData

,Stock,Date,Video ID,View,Like,Dislike,Comment
0,,,,NaN,NaN,NaN,NaN
1,,,,NaN,NaN,NaN,NaN
2,,,,NaN,NaN,NaN,NaN
3,,,,NaN,NaN,NaN,NaN
4,,,,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
95,,,,NaN,NaN,NaN,NaN
96,,,,NaN,NaN,NaN,NaN
97,,,,NaN,NaN,NaN,NaN
98,,,,NaN,NaN,NaN,NaN


In [17]:
for i in range(len(YoutubeStatData)):
    #nan인 경우 제거
    if(YoutubeStatData['View'][i] != YoutubeStatData['View'][i]):
        YoutubeStatData.drop(index=i, axis=0,inplace = True)
    #조회수 1만 이상을 만족하지 않는 동영상 지표는 제거하기     
    else:
        if(int(YoutubeStatData['View'][i])<10000):
            YoutubeStatData.drop(index=i, axis=0,inplace = True)
YoutubeStatData

,Stock,Date,Video ID,View,Like,Dislike,Comment


In [18]:
#csv로 저장하기
YoutubeStatData.to_csv("1.csv",index=False)

In [13]:
data1 = pd.read_csv("1.csv")
data2 = pd.read_csv("2.csv")
data3 = pd.read_csv("3.csv")
result = pd.concat([data1,data2,data3])
result

,Stock,Date,Video ID,View,Like,Dislike,Comment
0,카카오,2020-04-05,HClI4o3P7eo,10225.0,532.0,8.0,83.0


In [14]:
result.to_csv("result04.csv",index=False)

In [19]:
#데이터 합산
data1 = pd.read_csv("result01.csv")
data2 = pd.read_csv("result02.csv")
data3 = pd.read_csv("result03.csv")
data4 = pd.read_csv("result04.csv")
data5 = pd.read_csv("result05.csv")
data6 = pd.read_csv("result06.csv")
data7 = pd.read_csv("result07.csv")
data8 = pd.read_csv("result08.csv")
data9 = pd.read_csv("result09.csv")
data10 = pd.read_csv("result10.csv")
data11 = pd.read_csv("result11.csv")
data12 = pd.read_csv("result12.csv")
result = pd.concat([data1,data2,data3,data4,data5,data6,data7,data8,data9,data10,data11,data12])
result

,Stock,Date,Video ID,View,Like,Dislike,Comment
0,카카오,2020-07-03,GAj_h1lhoTA,42481.0,1016.0,36.0,29.0
1,카카오,2020-07-04,GYUUCCJL5FQ,16937.0,1035.0,9.0,29.0
2,카카오,2020-07-06,Gy-3SyegrWc,42357.0,586.0,17.0,71.0
3,카카오,2020-07-06,k36dPQ8doXo,16298.0,126.0,2.0,22.0
4,카카오,2020-07-07,WsCFG4Agysw,11332.0,646.0,15.0,153.0
...,...,...,...,...,...,...,...
11,카카오,2020-05-26,P2dePeqhi74,18530.0,1236.0,66.0,212.0
12,카카오,2020-05-27,Le3_yfGoAzw,34812.0,1140.0,42.0,182.0
13,카카오,2020-05-27,0VVgzJGVXHM,20940.0,1484.0,43.0,189.0
14,카카오,2020-05-28,k2kP39J7NlM,73156.0,1207.0,34.0,85.0


In [20]:
result.to_csv("kakao.csv",index=False)

###### 우선은 #1의 영상 데이터 수집 과정까지만 진행하기 (20/8~21/7 1만 조회수 이상의 영상 정보 수집)
#2 요일별 지표 합산 형태로 데이터 재가공

In [18]:
#조건을 만족하는 동영상 수를 확인하기 위한 작업
analysis = YoutubeStatData.groupby(['Stock','Date']).describe()
analysis['View']['count'][0]

6.0

In [19]:
#요일별 합계 지표 저장 작업
final = YoutubeStatData.groupby(['Stock','Date']).sum()
final

View   Like  Dislike  Comment
Stock Date                                       
카카오   2021-07-01  299723  10361      190      406
      2021-07-02  302506  11453      271      888
      2021-07-03   83555   2446       55      160
      2021-07-04  260130   7160      109      426
      2021-07-05  199598   6121       92      270
      2021-07-06   63586   2021       24      153
      2021-07-07   13266    605        6       12
      2021-07-08   60486   2688       46      131
      2021-07-09  132509   5074       85      380
      2021-07-10  176613   6803      158      283

In [105]:
#search 데이터 원래값으로 복원하고 1만 이상 조회수를 가진 동영상 갯수 지표 추가하기
final['Video Count'] = 0
final.astype({'Video Count': np.int64})

for i in range(len(analysis)):
    final['Video Count'][i] = int(analysis['View']['count'][i])
final

View   Like  Dislike  Comment  Search  Video Count
Stock Date                                                            
카카오   2021-07-16  160758   2330       52      181    7760            3
      2021-07-17  371651  15261      254      446    2014            7
      2021-07-18   48437   1997       36      102    1765            2
      2021-07-19   98743   4286       90      257    7760            3
      2021-07-20  213816   7261      227      386    8988            9

In [106]:
#csv로 저장하기
final.to_csv("result.csv",index=False)